# Install dependencies

In [ ]:
!pip install langchain-openai langchain langchain-community
!pip install faiss-cpu
!pip install sentence_transformers

# Run OpenAI compatible LLM server

Execute `python -m llama_cpp.server --model <path-to-your-model>` 

For the tutorial I've used Llama 2 7b chat quantize model which you can find on HuggingFace:

https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF

# Creating the knowledge base

Here we mock up a knowledge base by loading some information from a web page 

In [35]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://godofwar.fandom.com/wiki/Mimir")
documents = loader.load()

## Preprocess the text

Split the text into smaller chunks which represent the documents of the knowledge base

In [36]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

Created a chunk of size 832, which is longer than the specified 300
Created a chunk of size 407, which is longer than the specified 300
Created a chunk of size 420, which is longer than the specified 300
Created a chunk of size 5162, which is longer than the specified 300
Created a chunk of size 18039, which is longer than the specified 300
Created a chunk of size 3824, which is longer than the specified 300
Created a chunk of size 3224, which is longer than the specified 300
Created a chunk of size 1666, which is longer than the specified 300
Created a chunk of size 7296, which is longer than the specified 300
Created a chunk of size 4208, which is longer than the specified 300
Created a chunk of size 8208, which is longer than the specified 300
Created a chunk of size 1796, which is longer than the specified 300


# Creating the vector store

Create vectors from the documents using an embeddings model

In [37]:

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
db = FAISS.from_documents(docs, embeddings)

# Retrieval Augmented Generation

Let's use the OpenAI compatible LLM server to generate output using RAG

In [39]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo", 
                  openai_api_base="http://localhost:8000/v1",
                  openai_api_key="sk-xxx", 
                  max_tokens=2048,
                  temperature=0.7)

In [42]:
question = "What is the other name of Mimir?"
docs = db.similarity_search(question)
print(docs[0].page_content)

Mimir (Nordic: ᛗᛁᛗᛁᚱ), formerly known as Puck, is a Celtic fae who became Odin's advisor and the ambassador of the Aesir Gods until Odin imprisoned him for 109 years. After being freed by Kratos and Atreus, he became their ally. He is the tritagonist of God of War (2018) and God of War Ragnarök.


## RAG based model prompting

In [43]:
from langchain.schema import HumanMessage, SystemMessage

messages_rag = [
    SystemMessage(

        content=f"""You are a helpful assistant that answers questions based on a given context. You use only the current context for your answer. If the question cannot be answered directly from the context then output "N/A".
                    Current context:
                    {docs[0].page_content}
                    """
    ),
    HumanMessage(content=question)
]

In [44]:
response = chat.invoke(messages_rag)
print(response.content.lstrip())

Based on the provided context, the other name of Mimir is Puck.


## Normal model prompting (non RAG)

In [45]:
messages_non_rag = [
    SystemMessage(
        content=f"""You are a helpful assistant that answers questions."""
    ),
    HumanMessage(content=question),
]

In [46]:
response = chat.invoke(messages_non_rag)
print(response.content.lstrip())

Ah, an excellent question! Mimir is also known as Mim to some people. It's a common nickname or shortened form of the name Mimir. So, you can use either "Mimir" or "Mim" interchangeably when referring to this fascinating being from Norse mythology!
